# Updates on the Chunking Algorithm
This notebook is for the blog on improvements to our chunking algorithm.

In [65]:
import re
from tree_sitter_languages import get_language, get_parser

language = get_language('python')
parser = get_parser('python')

## Meet the Span

In [140]:
from __future__ import annotations
from dataclasses import dataclass

@dataclass
class Span:
    # Represents a slice of a string
    start: int = 0
    end: int = 0

    def __post_init__(self):
        # If end is None, set it to start
        if self.end is None:
            self.end = self.start

    def extract(self, s: str) -> str:
        # Grab the corresponding substring of string s by bytes
        return s[self.start: self.end]

    def extract_lines(self, s: str) -> str:
        # Grab the corresponding substring of string s by lines
        return "\n".join(s.splitlines()[self.start:self.end])

    def __add__(self, other: Span | int) -> Span:
        # e.g. Span(1, 2) + Span(2, 4) = Span(1, 4) (concatenation)
        # There are no safety checks: Span(a, b) + Span(c, d) = Span(a, d)
        # and there are no requirements for b = c.
        if isinstance(other, int):
            return Span(self.start + other, self.end + other)
        elif isinstance(other, Span):
            return Span(self.start, other.end)
        else:
            raise NotImplementedError()

    def __len__(self) -> int:
        # i.e. Span(a, b) = b - a
        return self.end - self.start

The example code we're gonna use in this guide will be from https://github.com/sweepai/sweep/blob/b267b613d4c706eaf959fe6789f11e9a856521d1/sweepai/handlers/on_check_suite.py, our old handler for parsing GitHub Action run logs at Sweep.

In [158]:
import requests

example_file = "https://raw.githubusercontent.com/sweepai/sweep/b267b613d4c706eaf959fe6789f11e9a856521d1/sweepai/handlers/on_check_suite.py"
python_code = requests.get(example_file).text

Let's first visualize the syntax tree.

In [159]:
tree = parser.parse(python_code.encode("utf-8"))

def pretty_node(node):
    return f"{node.type}:{node.start_byte}-{node.end_byte}"

def print_tree(node, indent=""):
    if len(re.sub("\s", "", node.text.decode("utf-8"))) < 100:
        return
    print(indent + pretty_node(node))
    for child in node.children:
        print_tree(child, indent=indent + "  ")

for child in tree.root_node.children:
    print_tree(child)

expression_statement:431-695
  assignment:431-695
    string:445-695
function_definition:697-1501
  block:776-1501
    expression_statement:776-952
      assignment:776-952
        dictionary:786-952
    expression_statement:957-1102
      assignment:957-1102
        call:968-1102
    if_statement:1126-1481
      block:1166-1399
        for_statement:1231-1399
function_definition:1504-2106
  block:1539-2106
    expression_statement:1642-1993
      assignment:1642-1993
        list:1653-1993
function_definition:2109-4425
  block:2161-4425
    if_statement:2354-2511
      block:2392-2511
    if_statement:3211-3396
      block:3257-3396
        expression_statement:3257-3396
          augmented_assignment:3257-3396
    if_statement:3446-3858
      block:3562-3858
        expression_statement:3562-3745
          assignment:3562-3745
            call:3572-3745
              argument_list:3600-3745
                binary_operator:3601-3744
    expression_statement:3983-4396
      call:3983-4

We can see that it doesn't actually line up:

```python
expression_statement:432-696
  assignment:432-696
    string:446-696
function_definition:698-1502
  block:777-1502
    expression_statement:777-953
      assignment:777-953
        dictionary:787-953
```

Notice that the “expression_statement” ends on byte 696 and “function_definition” starts on byte 698, skipping a byte.

In [67]:
def connect_chunks(chunks: list[Span]):
    for prev, curr in zip(chunks[:-1], chunks[1:]):
        prev.end = curr.start
    return chunks

## Coalescing

Here was the algo presented in the last blog. Unfortunately it has some bugs.

In [156]:
from tree_sitter import Node
from dataclasses import field

def chunk_node(
    node: Node, 
    text: str, 
    MAX_CHARS: int = 600
) -> list[str]:
    chunks = []
    current_chunk = ""
    for child in node.children:
        if child.end_byte - child.start_byte > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = ""
            chunks.extend(chunk_node(child, text, MAX_CHARS))
        elif child.end_byte - child.start_byte + len(current_chunk) > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = text[child.start_byte: child.end_byte]
        else:
            current_chunk += text[child.start_byte: child.end_byte]
    chunks.append(current_chunk)

    return chunks

for chunk in chunk_node(tree.root_node, python_code):
    print(chunk + "\n\n====================\n\n")

import ioimport osimport zipfileimport openaiimport requestsfrom loguru import loggerfrom sweepai.core.gha_extraction import GHAExtractorfrom sweepai.events import CheckRunCompletedfrom sweepai.handlers.on_comment import on_commentfrom sweepai.utils.config.client import SweepConfig, get_gha_enabledfrom sweepai.utils.github_utils import get_github_client, get_tokenopenai.api_key = os.environ.get("OPENAI_API_KEY")



log_message = """GitHub actions yielded the following error. 

{error_logs}

This is likely a linting or type-checking issue with the source code but if you are updating the GitHub Actions or versioning, this could be an issue with the GitHub Action yaml files."""



defdownload_logs(repo_full_name: str, run_id: int, installation_id: int):



headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {get_token(installation_id)}",
        "X-GitHub-Api-Version": "2022-11-28"
    }response = requests.get(f"https://api.github.com/repos/{repo

Here it is with the fixes by using the start_byte of the next node instead of the end_byte of the current node. 

I added a fake node at the end with start and end bytes equal to the end byte of the entire node. This is so that we don't need to rewrite the loop logic one last time for the last node. The purpose of MockNode is because the tree_sitter Node library doesn't have a constructor.

In [130]:
@dataclass
class MockNode:
    start_byte: int = 0
    end_byte: int = 0
    children: list[MockNode] = field(default_factory=list)

def chunk_node(
    node: Node, 
    text: str, 
    MAX_CHARS: int = 600
) -> list[str]:
    chunks = []
    current_chunk = ""
    node_children = node.children + [MockNode(node.end_byte, node.end_byte)]

    for child, next_child in zip(node_children[:-1], node_children[1:]):
        if child.end_byte - child.start_byte > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = ""
            chunks.extend(chunk_node(child, text, MAX_CHARS))
        elif child.end_byte - child.start_byte + len(current_chunk) > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = text[child.start_byte: next_child.start_byte]
        else:
            current_chunk += text[child.start_byte: next_child.start_byte]
    chunks.append(current_chunk)

    return chunks

for chunk in chunk_node(tree.root_node, python_code):
    print(chunk + "\n\n====================\n\n")

import io
import os
import zipfile

import openai
import requests
from loguru import logger

from sweepai.core.gha_extraction import GHAExtractor
from sweepai.events import CheckRunCompleted
from sweepai.handlers.on_comment import on_comment
from sweepai.utils.config.client import SweepConfig, get_gha_enabled
from sweepai.utils.github_utils import get_github_client, get_token

openai.api_key = os.environ.get("OPENAI_API_KEY")





log_message = """GitHub actions yielded the following error. 

{error_logs}

This is likely a linting or type-checking issue with the source code but if you are updating the GitHub Actions or versioning, this could be an issue with the GitHub Action yaml files."""





def download_logs(repo_full_name: str, run_id: int, installation_id: int):
    



headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {get_token(installation_id)}",
        "X-GitHub-Api-Version": "2022-11-28"
    }
    response = requests.get(f"https

Firstly, using Span's we can clean up the code a bit. Like removing the MockNode altogether.

In [131]:
def chunk_node(
    node: Node, 
    MAX_CHARS: int = 600,
) -> list[Span]:
    chunks: list[Span] = []
    current_chunk: Span = Span(node.start_byte, node.start_byte)
    node_children = node.children
    for child in node_children:
        if child.end_byte - child.start_byte > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = Span(child.end_byte, child.end_byte)
            chunks.extend(chunk_node(child, MAX_CHARS))
        elif child.end_byte - child.start_byte + len(current_chunk) > MAX_CHARS:
            chunks.append(current_chunk)
            current_chunk = Span(child.start_byte, child.end_byte) 
        else:
            current_chunk += Span(child.start_byte, child.end_byte)
    chunks.append(current_chunk)
    return chunks

for chunk in chunk_node(tree.root_node):
    print(chunk)

Span(start=1, end=430)
Span(start=432, end=696)
Span(start=698, end=772)
Span(start=777, end=1122)
Span(start=1127, end=1502)
Span(start=1502, end=1502)
Span(start=1502, end=1502)
Span(start=1505, end=2107)
Span(start=2107, end=2107)
Span(start=2110, end=2157)
Span(start=2162, end=2759)
Span(start=2764, end=3207)
Span(start=3212, end=3442)
Span(start=3447, end=3979)
Span(start=3984, end=4426)
Span(start=4426, end=4426)
Span(start=4426, end=4426)


## Skipping Whitespace when Measuring Length

Gives heavily indented code the same number of lines per code.

In [152]:
def char_len(s: str) -> int: # old len function
	return len(s)

def non_whitespace_len(s: str) -> int: # new len function
    return len(re.sub("\s", "", s))

## Coalescing Chunks

Combining smaller chunks with larger ones.

In [148]:
def coalesce_chunks(chunks: list[Span], source_code: str, coalesce: int = 50) -> list[Span]:
    new_chunks = []
    current_chunk = Span(0, 0)
    for chunk in chunks:
        current_chunk += chunk
        if len(current_chunk) > coalesce and "\n" in current_chunk.extract(source_code):
            new_chunks.append(current_chunk)
            current_chunk = Span(chunk.end, chunk.end)
    if len(current_chunk) > 0:
        new_chunks.append(current_chunk)
    return new_chunks

for chunk in coalesce_chunks(chunk_node(tree.root_node), python_code):
    print(chunk.extract(python_code))


import io
import os
import zipfile

import openai
import requests
from loguru import logger

from sweepai.core.gha_extraction import GHAExtractor
from sweepai.events import CheckRunCompleted
from sweepai.handlers.on_comment import on_comment
from sweepai.utils.config.client import SweepConfig, get_gha_enabled
from sweepai.utils.github_utils import get_github_client, get_token

openai.api_key = os.environ.get("OPENAI_API_KEY")


log_message = """GitHub actions yielded the following error. 

{error_logs}

This is likely a linting or type-checking issue with the source code but if you are updating the GitHub Actions or versioning, this could be an issue with the GitHub Action yaml files."""


def download_logs(repo_full_name: str, run_id: int, installation_id: int):

    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {get_token(installation_id)}",
        "X-GitHub-Api-Version": "2022-11-28"
    }
    response = requests.get(f"https://api.g

## Use Line Numbers

Using line numbers instead of character indices. Works because Span is unit-agnostic.

In [151]:
def get_line_number(index: int, source_code: str) -> int:
    total_chars = 0
    for line_number, line in enumerate(source_code.splitlines(keepends=True), start=1):
        total_chars += len(line)
        if total_chars > index:
            return line_number - 1
    return line_number

for i, chunk in enumerate(coalesce_chunks(chunk_node(tree.root_node), python_code)):
    print(f"Chunk {i}: {get_line_number(chunk.start, python_code)}-{get_line_number(chunk.end, python_code)}")

Chunk 0: 0-15
Chunk 1: 15-21
Chunk 2: 21-23
Chunk 3: 23-32
Chunk 4: 32-41
Chunk 5: 41-61
Chunk 6: 61-75
Chunk 7: 75-87
Chunk 8: 87-90
Chunk 9: 90-96
Chunk 10: 96-109


## Final New Algorithm

Putting it altogether (switched back to MAX_CHARS=1500):

In [155]:
from tree_sitter import Tree

def chunker(
	tree: Tree,
	source_code: bytes,
	MAX_CHARS=512 * 3,
	coalesce=50 # Any chunk less than 50 characters long gets coalesced with the next chunk
) -> list[Span]:

    # 1. Recursively form chunks based on the last post (https://docs.sweep.dev/blogs/chunking-2m-files)
    def chunk_node(node: Node) -> list[Span]:
        chunks: list[Span] = []
        current_chunk: Span = Span(node.start_byte, node.start_byte)
        node_children = node.children
        for child in node_children:
            if child.end_byte - child.start_byte > MAX_CHARS:
                chunks.append(current_chunk)
                current_chunk = Span(child.end_byte, child.end_byte)
                chunks.extend(chunk_node(child))
            elif child.end_byte - child.start_byte + len(current_chunk) > MAX_CHARS:
                chunks.append(current_chunk)
                current_chunk = Span(child.start_byte, child.end_byte) 
            else:
                current_chunk += Span(child.start_byte, child.end_byte)
        chunks.append(current_chunk)
        return chunks
    chunks = chunk_node(tree.root_node)

    # 2. Filling in the gaps
    for prev, curr in zip(chunks[:-1], chunks[1:]):
        prev.end = curr.start
    curr.start = tree.root_node.end_byte

    # 3. Combining small chunks with bigger ones
    new_chunks = []
    current_chunk = Span(0, 0)
    for chunk in chunks:
        current_chunk += chunk
        if non_whitespace_len(current_chunk.extract(source_code)) > coalesce \
            and "\n" in current_chunk.extract(source_code):
            new_chunks.append(current_chunk)
            current_chunk = Span(chunk.end, chunk.end)
    if len(current_chunk) > 0:
        new_chunks.append(current_chunk)

    # 4. Changing line numbers
    line_chunks = [Span(get_line_number(chunk.start, source_code),
                    get_line_number(chunk.end, source_code)) for chunk in new_chunks]

    # 5. Eliminating empty chunks
    line_chunks = [chunk for chunk in line_chunks if len(chunk) > 0]

    return line_chunks

for chunk in chunker(tree, python_code):
    print(chunk.extract_lines(python_code) + "\n\n====================\n\n")


import io
import os
import zipfile

import openai
import requests
from loguru import logger

from sweepai.core.gha_extraction import GHAExtractor
from sweepai.events import CheckRunCompleted
from sweepai.handlers.on_comment import on_comment
from sweepai.utils.config.client import SweepConfig, get_gha_enabled
from sweepai.utils.github_utils import get_github_client, get_token

openai.api_key = os.environ.get("OPENAI_API_KEY")

log_message = """GitHub actions yielded the following error. 

{error_logs}

This is likely a linting or type-checking issue with the source code but if you are updating the GitHub Actions or versioning, this could be an issue with the GitHub Action yaml files."""

def download_logs(repo_full_name: str, run_id: int, installation_id: int):
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {get_token(installation_id)}",
        "X-GitHub-Api-Version": "2022-11-28"
    }
    response = requests.get(f"https://api.gith